## Домашнее задание по неделе 4

Как было рассказано на лекции, стохастический градиентый спуск сходится быстрее, чем полный, хотя и менее стабильно. В этом задании вам предлагается реализовать стохастический градиентный спуск и сравнить его с точным вычислением весов линейной модели по скорости работы и значению метрики качества.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

### Задание 0

Реализуйте класс ```LinearRegressionSGD``` c обучением и и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом.

In [ ]:
from sklearn.base import BaseEstimator

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4):
        """
        epsilon: разница для нормы изменения весов 
        max_steps: максимальное количество шагов в градиентном спуске
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []
    
    def fit(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: self
        """
        l, d = X.shape # l - кол-во объектов, d - признаков

        if self.w0 is None:
          self.w0 = np.zeros(d)

        self.w = self.w0

        for step in range(self.max_steps):
          self.w_history.append(self.w)
          w_new = self.w - self.alpha * self.calc_gradient(X, y)
          if np.linalg.norm(w_new - self.w) < self.epsilon:
            break
          self.w = w_new
        
        return self
    
    def predict(self, X):
        """
        X: np.array (l, d)
        ---
        output: np.array (l)
        """
        if self.w is None:
          raise Exception('Not trained yet')

        l, d = X.shape
        y_pred = []

        for i in range(l):
          y_pred.append(np.dot(X[i], self.w))
        
        return np.array(y_pred)
    
    def calc_gradient(self, X, y):
        """
        X: np.array (l, d)
        y: np.array (l)
        ---
        output: np.array (d)
        """
        l, d = X.shape
        gradient = []
        i = np.random.randint(0, l)

        for j in range(d):
          dL = 2/l * X[i][j] * (np.dot(self.w, X[i]) - y[i])
          gradient.append(dL)


        return np.array(gradient)

Проверять работу мы будем на имеющемся в sklearn наборе данных boston: в нём нужно по информации о доме предсказать его стоимость.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.3, random_state=10)

### Задание 1

Метрикой качества в нашей задаче будет MAPE - Mean Absolute Percentage Error. Реализуйте её с заданным интефейсом и вычислите 
```MAPE(y_test, y_0)```, где ```y_0 = (mean(y_test), mean(y_test), ...)```

In [ ]:
def MAPE(y_true, y_pred):
    """
        y_true: np.array (l)
        y_pred: np.array (l)
        ---
        output: float [0, +inf)
    """
    # mape = np.mean(np.abs((y_true - y_pred)/y_true)) * 100
    y_error = y_pred - y_true # рассчитайте вектор ошибок
    y_error_abs = [abs(i) for i in y_error] # рассчитайте вектор модуля ошибок
    perc_error_abs = y_error_abs/ y_true # рассчитайте вектор относительных ошибок
    mape = (sum(perc_error_abs) / len(y_true)) * 100
    return mape

In [ ]:
!pip show scikit-learn

Name: scikit-learn
Version: 0.24.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: scipy, numpy, threadpoolctl, joblib
Required-by: yellowbrick, umap-learn, textgenrnn, sklearn, sklearn-pandas, pynndescent, mlxtend, lucid, lightgbm, librosa, imbalanced-learn, fancyimpute


In [ ]:
import sklearn

y_0 = []
for i in range(len(y_test)):
  y_0.append(np.mean(y_test))

MAPE(y_test, y_0)
# sklearn.metrics.mean_absolute_percentage_error(y_test, y_0)

37.415882976840955

### Задание 2 

Обучите ```LinearRegressionSGD``` с базовыми параметрами на тренировочном наборе данных (```X_train```, ```y_train```), сделайте предсказание на тестовых данных ```X_test```, записав результат в переменную ```y_pred_sgd```  и вычислите ошибку MAPE.

In [ ]:
sgd = LinearRegressionSGD() # epsilon=1e-4/len(y_test), alpha=1e-4/len(y_test)

sgd.fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)

MAPE(y_test, y_pred_sgd)

38.20685540623259

### Задание 3

Вычислите веса по точной формуле, используя ```X_train``` и ```y_train```; предскажите с их помощью целевую переменную на ```X_test```, записав результат в переменную ```y_pred_lr``` и вычислите ошибку MAPE.

In [ ]:
w = np.dot(np.linalg.inv(np.dot(X_train.T, X_train)), (np.dot(X_train.T, y_train)))

y_pred_lr = np.dot(X_test, w) 

MAPE(y_test, y_pred_lr)

18.953134816376206